## Molecular dynamics using `covalent`

In this tutorial, we investigate how to build and run a simple MD simulation using `covalent`. For this example we simulate a 2D Lennard Jones melt (ideal gas) using the `Velocity-Verlet` time integration scheme.
As common in MD, we implement periodic boundary conditions along the `x` and `y` directions to effectively replicate an infinite system and use the `minimum image convention` when computing interaction forces.

![LennardJonesGas](./figures/lj_gas.png)

In this figure we can see the interaction neighbours of the red particle in focus as well as the periodic images of the unit cell in the middle. As the simulation evolves, the interaction neighbors of particles change as the move in and out of the unit cell. Given the periodic boundary conditions (PBCs) a particle leaving the unit cell from one edge is replaced by its periodic image from the opposite side. This method effectively simulates an infinite system of gas particles interacting via the 12-6 Lennard Jones pontetial given as follows

$V(r_{ij}) = 4 \epsilon \left[\left(\frac{\sigma}{r_{ij}}\right)^{12} - \left(\frac{\sigma}{r_{ij}}\right)^{6}\right]$

where $r_{ij} = r_{i} - r_{j}$ is the radial distance between particle $i$ and particle $j$ in the system. $\epsilon$ is the energy scale/strength of the potential and $\sigma$ is the length scale associate with $V(r)$.

![LJPotential](./figures/lj.jpg)

Image obtained from [here](https://www.google.com/search?q=lennard+jones+potential&client=firefox-b-d&sxsrf=APq-WBtQejymhmtDnrhxmVFqLt7p2EBo7Q:1650892658886&source=lnms&tbm=isch&sa=X&ved=2ahUKEwiW2fqzpq_3AhWNB80KHfm3DAsQ_AUoAXoECAEQAw&biw=1600&bih=739#imgrc=7Em1uCYew0-9oM)

From this image we can see that $\epsilon$ is the potential well depth and $\sigma$ is the length scale at which the potential cross the $x$ axis i.e $V(r) = 0$.


### Potential cutoff, $r_{c}$

In MD it is typical to include a `cutoff` distance beyond which two particles are considered to be non-interacting. In simulation this is implemented by simply making the potential piecewise using a `cutoff` parameter i.e

$V(r) = 4 \epsilon \left[\left(\frac{\sigma}{r_{ij}}\right)^{12} - \left(\frac{\sigma}{r_{ij}}\right)^{6}\right], r <= r_{c}$

$V(r) = 0, r > r_{c}$

Depending on what value of $r_c$ is chosen the LJ potential can be made attractive or repulsive. If we set the potential cutoff at the potential minimum i.e $\frac{\partial V}{\partial r} = 0$, we make the interactions purely repulsive. In the LJ case, this amounts to setting $r_{c} = 2^{1/6} \sigma$. Likewise, to get an attractive potential it is convention to place the cutoff at $r_c = 2.5\sigma$.


#### MD simulation steps

Having specified the interaction potential, the simulation is performed by simply integrating the classical Newton's equations of motion. The recipe of doing so is fairly straightforward and can be performed as per the following steps

* Compute forces, $F$ at timestep $t$ on all particles using their current positions, $r(t)$
* Perform a partial update of the velocities and a full update of the particle positions as follows

$\mathbf{v^{*}} = v(t) + \frac{\delta t}{2 m} \mathbf{F}(t)$

$\mathbf{r}(t + \delta t) = \mathbf{r}(t) + \mathbf{v^{*}}(t) \delta t$

* Compute forces on the particles again using the updates positions, $\mathbf{r}(t + \delta t)$

* Update velocities using the computed forces, $\mathbf{F}(\mathbf{r}(t + \delta))$

$\mathbf{v}(t + \delta t) = \mathbf{v}^{*}(t) + \frac{\delta t}{2 m} \mathbf{F}(\mathbf{r}(t + \delta t))$


#### Tutorial details

In this tutorial, we simulate a 2-D Lennard-Jones system with $N$ gas particles. Particles in our simulation are represented via the `Particle` class that contains its mass, position(s), velocity and forces as attributes. The `SimulationBox` class defines the domain within which the simulaton takes place and has the box dimension as its attributes i.e

* `xlo`: Lower edge of the box's extent in the $x$ direction

* `xhi`: Upper edge of the box's extent in the $x$ direction

* `ylo`: Lower edge of the box's extent in the $y$ direction

* `yhi`: Upper edge of the box's extent in the $y$ direction

* `@property lx`: Length of the box in the $x$ direction

* `@property ly`: Length of the box in the $y$ direction


We abstract the interaction potential via the `Potential` class that is basically is a `Callable` that takes in a position value as an input argument.

To successfully run this tutorial, we need to ensure `covalent` is successfully installed along with a few extra python packages

* `covalent`
* `numpy`
* `scipy`
* `matplotlib`

The easiest way to do so will be to create a python virtual environment using `conda` or any other means comfortable for the user

* `conda create -n covalent-md python=3.8`
* `conda activate covalent-md`
* `conda install numpy scipy matplotlib pandas`

In [1]:
from __future__ import annotations
import covalent as ct
import numpy as np
from typing import Tuple, List
from scipy.stats import norm
from scipy.optimize import minimize, Bounds
import pandas as pd
from pydantic import BaseModel
import time

In [2]:
class SimulationBox(object):
    """
    2-D Simulation domain
    """
    def __init__(self, xlo: float, xhi: float, ylo: float, yhi: float):
        self.xlo: float = xlo
        self.xhi: float = xhi
        self.ylo: float = ylo
        self.yhi: float = yhi

    @property
    def lx(self):
        return self.xhi - self.xlo

    @property
    def ly(self):
        return self.yhi - self.ylo

    def __str__(self):
        return f"SimulationBox(xlo={self.xlo}, xhi={self.xhi}, ylo={self.ylo}, yhi={self.yhi})"

class Particle(BaseModel):
    """
    Base simulation particle

    id: Unique ID of the particle
    mass: Mass of the particle
    x: x coordinate of the particle in the simulation domain
    y: y coordinate of the particle in the simulation domain
    vx: x component of the velocity of the particle
    vx: y component of the velocity of the particle
    fx: x component of the force acting on the particle
    fy: y component of the force acting on the particle
    """
    id: int
    mass: float = 1.0
    x: float
    y: float
    vx: float = 0.0
    vy: float = 0.0
    fx: float = 0.0
    fy: float = 0.0

    def __eq__(self, other: Particle):
        """
        Check if particles are the same by checking their id and mass attributes
        """
        return self.id == other.id and self.mass == other.mass

    def __str__(self):
        """
        Print particle
        """
        return f"{self.id},{self.mass},{self.x},{self.y},{self.vx},{self.vy},{self.fx},{self.fy}"

class Potential(object):
    """
    Base potential class that can be subclassed to define different pair potential types used in a simulation
    """
    def __init__(self):
        pass
    
    def __call__(self):
        """
        Evaluates the potential
        """
        raise NotImplementedError

    def force(self, x: float):
        """
        Returns the force dervied from the potential at position x
        """
        raise NotImplementedError

class LennardJones(Potential):
    def __init__(self, epsilon: float, sigma: float, cutoff: float):
        self.epsilon = epsilon  # Interaction strength
        self.sigma = sigma      # Interaction length scale
        self.cutoff = cutoff    # Interaction cutoff

    def __call__(self, r: float):
        if r <= self.cutoff:
            return 4*self.epsilon*((self.sigma/r)**12 - (self.sigma/r)**6)
        else:
            return 0.0

    def force(self, r: float):
        """
        LJ force at position r
        """
        if r <= self.cutoff:
            return -24*self.epsilon*((self.sigma**6/r**7) - 2*(self.sigma**12/r**13))
        else:
            return 0.0

We create simple helper methods to build and run the simulation step by step

In [13]:
def create_particle(id: int, mass: float, box: SimulationBox):
    """
    Return a new particle randomly located within the simulation box
    The velocity of the particle is randomly initialized from a normal distribution
    """
    x = np.random.uniform(box.xlo, box.xhi)
    y = np.random.uniform(box.ylo, box.yhi)
    vx = 0.0
    vy = 0.0
    return Particle(id=id, mass=mass, x=x, y=y, vx=vx, vy=vy)


def get_coordinates(particles: List[Particle]):
    """
    Converts a list of particles into a numpy array with particle coordinates
    """
    return np.array([val for p in particles for val in [p.x, p.y]])

def get_velocities(particles: List[Particle]):
    """
    From a list of particles returns a numpy array of particle velocities
    """
    return np.array([[p.vx, p.vy] for p in particles])

def get_particles_from_coordinates(x: List[float]) -> List[Particle]:
    """
    Returns a list of particles from the input numpy array of particle coordinates

    Each element of the array is a tuple for the corresponding particle's coordinate
    """
    # Create tuples from the list
    cpairs = [[x[i], x[i+1]] for i in range(0, len(x), 2)]
    # recreate particles from the list of coordinates
    return [Particle(id=i, mass=1.0, x=c[0], y=c[1]) for i, c in enumerate(cpairs)]

def dump(dump_id:int, particles: List[Particle], prefix: str):
    """
    Save a snapshot of the particles to disk in file `file`
    """
    with open(f"{prefix}-{dump_id}.txt", "w") as f:
        for p in particles:
            f.write(str(p)+"\n")
    f.close()

To simplify computing the total potential energy of the system, we define a function `compute_total_pe` that takes in the following arguments

* Coordinates of all the particles in the system as a numpy array
* `Potential` object to be used to compute the interaction energy between two particles

In [7]:
def check_minimum_image(x: float, L: float):
    """
    Check to ensure that the particle is interacting with its minimum image neighbour
    """
    if (x >= 0.5*L):
        return x - L
    elif (x < -0.5*L):
        return x + L
    else:
        return x

In [8]:
class Simulation(object):
    """
    Base simulation class that contains contains all the particles, domain and interaction potential
    """
    def __init__(self, timestep: float, particles: List[Particle], box: SimulationBox, potential: Potential):
        self.particles: List[Particle] = particles
        self.box = box
        self.potential = potential
        self.dt = timestep

    @ct.electron
    def _compute_forces(self):
        for i in self.particles:
            i.fx = 0.0
            i.fy = 0.0

            i.x = pbc(i.x, self.box.lx)
            i.y = pbc(i.y, self.box.ly)

            for j in self.particles:
                if i == j:
                    continue
                else:
                    j.x = pbc(j.x, self.box.lx)
                    j.y = pbc(j.y, self.box.ly)

                    dx = i.x - i.x
                    dy = i.y - i.y

                    if (dx > 0.5*self.box.lx):
                        dx = dx - self.box.lx
                    elif (dx <= -0.5*self.box.lx):
                        dx = dx + self.box.lx

                    if (dy > 0.5*self.box.ly):
                        dy = dy - self.box.ly
                    elif (dy <= -0.5*self.box.ly):
                        dy = dy + self.box.ly

                    dr = np.sqrt(dx**2 + dy**2)

                    i.fx += self.potential.force(dr)
                    i.fy += self.potential.force(dr)

    @ct.electron
    def _update_positions(self, step: int):
        org_particles = [p for p in self.particles]
        if step == 0:
            for p in self.particles:
                p.x = p.x + p.vx*self.dt + 0.5*p.fx*self.dt**2
                p.y = p.y + p.vy*self.dt + 0.5*p.fy*self.dt**2
        else:
            for i, p in enumerate(self.particles):
                p.x = 2*p.x - org_particles[i].x + p.fx*self.dt**2
                p.y = 2*p.y - org_particles[i].y + p.fy*self.dt**2

    @ct.electron
    def _dump(self, filename: str):
        with open(filename, "w") as f:
            for p in self.particles:
                f.write(f"{p.id}, {p.x}, {p.y}, {p.fx}, {p.fy}\n")
            f.close()

    #@ct.lattice
    def run(self, nsteps: int):
        for i in range(nsteps):
            self._compute_forces()
            self._update_positions()
            self._dump(f"simulation-{i}.txt")

### Simulation parameters

In [9]:
N = 10 # number of particles

# Potential paramters
epsilon = 1.0
sigma = 1.0
cutoff = 2.5*sigma

# Number of simulation steps
Nsteps = 50


# Simulation timestep
dt = 1e-4

# Simulation box/domain
xlo = 0
xhi = 5
ylo = 0
yhi = 5

# LJ potential
ljpot = LennardJones(epsilon=epsilon, sigma=sigma, cutoff=cutoff)

### Create simulation box

In [10]:
box = SimulationBox(xlo=xlo, xhi=xhi, ylo=ylo, yhi=yhi)

#### Create particles randomly distribute within the box

In [11]:
particles = [create_particle(id=i, mass=1.0, box=box) for i in range(N)]

In MD simulations, it is typical to minimize the potential energy of the system before starting a simulation. To this end, we minimize the total PE of the system of particles just created using `minimize` function from `scipy.optimize` module

In [12]:
def total_pe(coordinates: List[float], potential: Potential, box: SimulationBox):
    """
    From the input configuration of particles, compute the total potential energy of the system
    """
    df = pd.DataFrame([{"x": coordinates[i], "y": coordinates[i+1]} for i in range(0, len(coordinates), 2)])

    # Map all coordinates of the particles back into the simulation unit cell
    df = df.apply(lambda row: pd.Series({'x': row['x']%box.lx, 'y': row['y']%box.ly}), axis=1)

    # Loop over all particles in the dataframe, and from each compute the interaction energy of it with due to everything else
    total_pe_energy = 0.0
    for base_particle in df.iterrows():
        dr = df.apply(lambda row: np.sqrt((row['x'] - base_particle[1]['x'])**2 + (row['y'] - base_particle[1]['y'])**2), axis=1)
        pe = dr.apply(lambda r: ljpot(r) if r != 0.0 else None)
        total_pe_energy += pe.dropna().sum()

    return np.abs(total_pe_energy)

##### Relax system

In [22]:
num_iters = 0

def save_snap

res = minimize(total_pe, get_coordinates(particles), method='Nelder-Mead',
        args=(ljpot, box,),
        callback=lambda x: dump(num_iters)
        bounds=[val for i in range(N) for val in [(box.xlo, box.xhi), (box.ylo, box.yhi)]],
       tol=1e-12,
       options={'disp': True})

TypeError: <lambda>() missing 1 required positional argument: 'state'

In [20]:
total_pe(res.x, ljpot, box)

1.3125611708630913e-13

In [21]:
res.x

array([0.26777397, 0.5257359 , 3.48351911, 4.86543261, 4.11661374,
       1.34484836, 3.40002966, 3.53072918, 2.75369538, 4.23419657,
       4.20230196, 0.3102409 , 4.43662013, 3.31480517, 4.29858979,
       4.26954955, 1.34802152, 2.91193911, 4.43250013, 2.34488765])

### Map to discrete functions to enable mapping to a workflow

In [ ]:
@ct.electron
def pbc(x, L):
    if (x >= 0.5*L):
        return x - L
    elif (x < -0.5*L):
        return x + L
    else:
        return x

@ct.electron
def create_simulation_box(xlo: float, xhi: float, ylo: float, yhi: float):
    return SimulationBox(xlo, xhi, ylo, yhi)

@ct.electron
def create_particles(N: int, box: SimulationBox) -> List[Particle]:
    particles = []
    for i in range(N):
        mass = np.random.uniform(1, 2)
        xcor = np.random.uniform(box.xlo, box.xhi)
        ycor = np.random.uniform(box.ylo, box.yhi)
        vx = norm.rvs()
        vy = norm.rvs()
        particles.append(Particle(i, mass, xcor, ycor, vx, vy))
    return particles

@ct.electron
def compute_forces(particles: List[Particle], box: SimulationBox, potential: Potential):
    for i in particles:
        i.fx = 0.0
        i.fy = 0.0
        for j in particles:
            if i == j:
                continue
            else:
                dx = i.x - j.x
                dy = i.y - j.y

                dx = pbc(dx, box.lx)
                dy = pbc(dy, box.ly)

                i.fx += potential.force(dx)
                i.fy += potential.force(dy)
    return particles

@ct.electron
def dump(step: int, particles: List[Particle]):
    with open(f"simulation-{step}.txt", "w") as f:
        for p in particles:
            f.write(f"{str(p)}\n")
        f.close()

@ct.electron
def update_positions(step: int, dt: float, particles: List[Particle], box: SimulationBox, old_particles: List[Particle] = []):
    copy_particles: List[Particle] = []
    copy_particles = [p for p in particles]
    if step == 0:
        for p in particles:
            p.x = (p.x + p.vx*dt + (p.fx/p.mass)*dt**2)%box.lx
            p.y = (p.y + p.vy*dt + (p.fy/p.mass)*dt**2)%box.ly
    else:
        for index, p in enumerate(particles):
            p.x = (2*p.x - old_particles[index].x + (p.fx/p.mass)*dt**2)%box.lx
            p.y = (2*p.y - old_particles[index].y + (p.fy/p.mass)*dt**2)%box.ly
    
    return particles, copy_particles

@ct.electron
@ct.lattice
def integrate(step: int, dt: float, particles: List[Particle], box: SimulationBox, potential: Potential):
    old_particles: List[Particle] = []
    old_particles = [p for p in particles]
    particles = compute_forces(particles=particles, box=box, potential=potential)
    particles, old_particles = update_positions(step=step, dt=dt, particles=particles, box=box, old_particles=old_particles)
    dump(step=step, particles=particles)
    return particles, old_particles

@ct.electron
def get_particle_coordinates(particles: List[Particle])-> List[float]:
    return [c for p in particles for c in [p.x, p.y]]

# Total PE of the system
@ct.electron
def total_pe(x: List[float], box: SimulationBox, potential: Potential, N: int):
    total_pe = 0.0
    # recreate particles from the list of coordinates
    coordinates = [(x[i]%box.lx, x[i+1]%box.ly) for i in range(0, 2*N, 2)]
    particles = [Particle(i, 1.0, c[0], c[1]) for i, c in enumerate(coordinates)]

    for i in particles:
        i.x = i.x
        i.y = i.y
        for j in particles:
            if i == j:
                continue
            else:
                dx = i.x - j.x
                dy = i.y - j.y

                dx = pbc(dx, box.lx)
                dy = pbc(dy, box.ly)

                dr = np.sqrt(dx**2 + dy**2)
                total_pe += potential(dr)
    return total_pe

@ct.electron
def relax_system(particles: List[Particle], box: SimulationBox, potential: Potential):
    x0 = get_particle_coordinates(particles)
    res = minimize(total_pe, x0, args=(box, LJ(1.0, 1.0, 2.5), len(particles),), method="powell")
    coordinates = [[res.x[i], res.x[i+1]] for i in range(0, len(res.x), 2)]
    return [Particle(i, 1.0, c[0], c[1]) for i, c in enumerate(coordinates)]


#### Relax system

In [ ]:
@ct.lattice
def workflow(num_particles: int, nsteps: int, dt: float):
    sbox = create_simulation_box(xlo=0, xhi=20, ylo=0, yhi=20)
    particles = create_particles(N=num_particles, box=sbox)
    particles = relax_system(particles=particles, box=sbox, potential=LJ(1.0, 1.0, 2.5))
    #particles = [Particle(i, 1.0, c[0], c[1]) for i, c in enumerate(res.x)]
    for i in range(nsteps):
        particles, _ = integrate(step=i, dt=dt, particles=particles, box=sbox, potential=LJ(1.0, 1.0, 2.5))
    return particles

In [ ]:
dispatch_id = ct.dispatch(workflow)(10, 5, 1e-4)
print(dispatch_id)
#result = ct.get_result(dispatch_id=dispatch_id, wait=True)